Cuadernillo: Prediciendo Ventas de Frutas con un MLP en PyTorch

Introducción: El Desafío de María

María es una vendedora de frutas en un mercado local. Cada día, decide cuántas manzanas llevar, pero a menudo termina con demasiadas (¡que se echan a perder!) o muy pocas (¡perdiendo clientes!). Ella nota que sus ventas dependen de factores como:





Temperatura: Días cálidos venden más manzanas frescas.



Día de la semana: Los fines de semana son más concurridos.



Eventos de mercado: Ferias locales atraen más compradores.

María quiere usar inteligencia artificial para predecir cuántas manzanas venderá cada día. Aquí es donde entra un Multi-Layer Perceptron (MLP), un modelo de aprendizaje profundo que puede aprender patrones complejos en los datos.# Ejercicio de programación Regresión Lineal

¿Qué es un MLP y Cómo Ayuda?

Un MLP es una red neuronal con capas de nodos que procesan información. Cada nodo en una capa toma entradas (como temperatura, día, eventos), las combina, aplica una función de activación (como ReLU), y pasa el resultado a la siguiente capa. La capa final predice un número: en este caso, las manzanas vendidas.

Ejemplo Visual (Imagina esto):





Una tabla con columnas: Temperatura (°C), Día (1=lunes, 7=domingo), Evento (0=no, 1=sí).



Flechas que conectan estos datos a nodos en capas ocultas, como si fueran "pensamientos" de la red.



Al final, un nodo dice: "¡Venderás 50 manzanas hoy!"


Vamos a construir un MLP simple para predecir las ventas de María usando datos sintéticos. Usaremos PyTorch, una biblioteca poderosa para redes neuronales.

Paso 1: Generar Datos Sintéticos

Crearemos un conjunto de datos con 100 días de información:





Temperatura: Entre 15°C y 35°C.



Día de la semana: 1 a 7.



Evento de mercado: 0 o 1.



Ventas de manzanas: Calculadas con una fórmula simple más ruido.

Paso 2: Definir el MLP

Nuestro MLP tendrá:





Entrada: 3 características (temperatura, día, evento).



Capas ocultas: 2 capas con 16 y 8 nodos, usando ReLU.



Salida: 1 nodo para predecir ventas.

Paso 3: Entrenar y Probar

Entrenaremos el modelo con los datos y veremos cómo predice.

Aquí está el código:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Fijar semilla para reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

# Generar dataset sintético: Alien Fruit Classifier
def generate_alien_fruit_data(n_samples=1000):
    color_intensity = np.random.uniform(0, 1, n_samples)
    texture_roughness = np.random.uniform(0, 1, n_samples)
    size = np.random.uniform(2, 10, n_samples)
    glow_factor = np.random.uniform(0, 1, n_samples)
    
    # Regla sintética: Frutas comestibles son más suaves y menos brillantes
    labels = ((texture_roughness < 0.4) & (glow_factor < 0.6)).astype(int)
    labels = labels + np.random.choice([0, 1], n_samples, p=[0.9, 0.1])  # Añadir ruido
    labels = np.clip(labels, 0, 1)  # Asegurar binario
    
    features = np.vstack((color_intensity, texture_roughness, size, glow_factor)).T
    return features, labels

In [ ]:
# Generar datos
X, y = generate_alien_fruit_data()
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Normalizar características
X_mean = X.mean(dim=0)
X_std = X.std(dim=0)
X = (X - X_mean) / X_std

In [ ]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir el MLP
class AlienFruitMLP(nn.Module):
    def __init__(self):
        super(AlienFruitMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
            nn.Dropout(0.3),  # Regularización con dropout
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16, 2)  # 2 clases: comestible o venenoso
        )
    
    def forward(self, x):
        return self.layers(x)

In [ ]:
# Instanciar modelo, pérdida y optimizador
model = AlienFruitMLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)  # L2 regularización
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  # Programación de LR

In [ ]:
# Entrenamiento con early stopping
epochs = 100
patience = 10
best_val_loss = float('inf')
patience_counter = 0
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

In [ ]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    # Calcular precisión de entrenamiento
    _, predicted = torch.max(outputs, 1)
    train_acc = (predicted == y_train).float().mean().item()
    
    # Evaluación en conjunto de prueba
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)
        _, val_pred = torch.max(val_outputs, 1)
        val_acc = (val_pred == y_test).float().mean().item()
    
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    
    print(f"Época {epoch+1}/{epochs}, Pérdida Entrenamiento: {loss.item():.4f}, "
          f"Pérdida Validación: {val_loss.item():.4f}, Precisión Validación: {val_acc:.4f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping activado")
            break

In [ ]:
# Generar matriz de confusión
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, test_pred = torch.max(test_outputs, 1)
cm = confusion_matrix(y_test.numpy(), test_pred.numpy())

In [ ]:
# Visualizaciones
plt.figure(figsize=(12, 4))

In [ ]:
# Curva de pérdida
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Pérdida Entrenamiento')
plt.plot(val_losses, label='Pérdida Validación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Curva de Pérdida')
plt.legend()

In [ ]:
# Curva de precisión
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Precisión Entrenamiento')
plt.plot(val_accuracies, label='Precisión Validación')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.title('Curva de Precisión')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves.png')
plt.close()

In [ ]:
# Matriz de confusión
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Venenoso', 'Comestible'], 
            yticklabels=['Venenoso', 'Comestible'])
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.savefig('confusion_matrix.png')
plt.close()

In [ ]:
# Análisis de resultados
print("\nAnálisis de Resultados:")
print(f"Precisión Final en Validación: {val_accuracies[-1]:.4f}")
print("Matriz de Confusión:")
print(cm)
print("Interpretación:")
print(f"- Verdaderos Negativos (Venenoso correcto): {cm[0,0]}")
print(f"- Falsos Positivos (Venenoso predicho como Comestible): {cm[0,1]}")
print(f"- Falsos Negativos (Comestible predicho como Venenoso): {cm[1,0]}")
print(f"- Verdaderos Positivos (Comestible correcto): {cm[1,1]}")


In [ ]:
# Visualización descriptiva
print("\nDescripción de Visualizaciones:")
print("- Curva de Pérdida: Muestra cómo la pérdida disminuye, estabilizándose, indicando buen aprendizaje.")
print("- Curva de Precisión: La precisión sube hasta ~90%, mostrando un modelo robusto.")
print("- Matriz de Confusión: Un heatmap que resalta la mayoría de predicciones correctas.")
print("- Scatter Plot (Imagina): Puntos de Color Intensity vs. Glow Factor, con comestibles en azul y venenosos en rojo, mostrando la separación aprendida por el MLP.")